# Step 1 – Backfill

### Setting up Hopsworks

In [30]:
import hopsworks
import os
import datetime

In [31]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")

project = hopsworks.login()
print(project.description)

2025-01-07 10:49:41,585 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 10:49:41,598 INFO: Initializing external client
2025-01-07 10:49:41,598 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 10:49:42,859 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159321
Default project


### Getting historical data

In [32]:
import requests
import pandas as pd
import numpy as np
from functions import util
from importlib import reload

In [33]:
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
fixtures_url = "https://fantasy.premierleague.com/api/fixtures?futures=1"
player_details_url = "https://fantasy.premierleague.com/api/element-summary/{element_id}"
player_stats_gw_url = "https://fantasy.premierleague.com/api/event/{gw}/live/"

## Fetching general data from https://fantasy.premierleague.com/api/bootstrap-static/

### From the general data, we take genreal information about the players, teams, and events (gameweeks).

In [34]:
general_data = requests.get(bootstrap_url).json()

In [35]:
upcoming_fixtures_data = requests.get(fixtures_url).json()

# Backfill player stats for each game week

### Init vars

In [36]:
gameweek = 1
all_gameweek_data = []

In [37]:
upcoming_fixtures = pd.DataFrame(upcoming_fixtures_data)
upcoming_fixtures.dropna(subset=["event"], inplace=True)
upcoming_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
1,2444470,1.0,True,True,1,2024-08-16T19:00:00Z,90,False,True,9,0.0,14,1.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",3,3,115827
2,2444473,1.0,True,True,4,2024-08-17T11:30:00Z,90,False,True,12,2.0,10,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,115830
3,2444471,1.0,True,True,2,2024-08-17T14:00:00Z,90,False,True,20,0.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,5,115828
4,2444472,1.0,True,True,3,2024-08-17T14:00:00Z,90,False,True,5,3.0,8,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,3,115829
5,2444474,1.0,True,True,5,2024-08-17T14:00:00Z,90,False,True,17,0.0,15,1.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,4,115831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2444845,38.0,False,False,376,2025-05-25T15:00:00Z,0,False,False,8,NaN,15,NaN,[],2,4,116202
376,2444846,38.0,False,False,377,2025-05-25T15:00:00Z,0,False,False,6,NaN,16,NaN,[],4,3,116203
377,2444847,38.0,False,False,378,2025-05-25T15:00:00Z,0,False,False,1,NaN,17,NaN,[],4,2,116204
378,2444848,38.0,False,False,379,2025-05-25T15:00:00Z,0,False,False,5,NaN,18,NaN,[],3,4,116205


In [38]:
general_stats = pd.DataFrame(general_data["elements"])
general_stats

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,130,77,690,309,702,316,650,283,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-1,1,2,...,29,14,21,4,191,25,38,10,0.90,0.30
2,True,True,100.0,100.0,226597,0,0,4,-4,2,...,51,3,57,16,27,2,9,2,1.05,0.35
3,True,True,75.0,75.0,219847,0,0,-1,1,1,...,13,5,79,14,38,11,42,12,1.03,0.36
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,636,58,459,55,535,61,570,68,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,True,True,75.0,75.0,509291,0,0,0,0,0,...,222,140,185,86,314,142,593,248,1.20,0.18
707,True,True,NaN,NaN,463212,0,0,0,0,0,...,116,70,332,169,473,227,571,231,0.38,0.00
708,True,True,NaN,NaN,517179,0,0,-1,1,0,...,706,236,426,153,396,130,228,87,0.00,0.00
709,True,True,NaN,NaN,596054,0,0,0,0,0,...,464,293,623,264,671,296,627,267,0.00,0.00


### Loop through each gameweek and fetch player stats

### Finally concat into single dataframe

In [ ]:
while True:
    player_stats_gw_url_formatted = player_stats_gw_url.format(gw=gameweek)
    player_stats = requests.get(player_stats_gw_url_formatted).json()

    # Get the current gameweek from the helper function
    current_gameweek = util.get_gameweek_from_date(datetime.datetime.now())

    # Only fetch data if there are elements in the response and the gameweek is not in the future
    if len(player_stats["elements"]) == 0 or gameweek > current_gameweek:
        print(f"No more data available after gameweek {gameweek - 1}")
        break

    # Create a DataFrame from the player stats data
    player_stats_gw_df = pd.DataFrame(player_stats["elements"]).drop(columns=["explain", "modified"])
    player_stats_gw_df = pd.concat(
        [player_stats_gw_df.drop(columns=["stats"]), player_stats_gw_df["stats"].apply(pd.Series)],
        axis=1
    )

    player_stats_gw_df["gameweek"] = gameweek

    all_gameweek_data.append(player_stats_gw_df)

    print(f"Fetched data for gameweek {gameweek}")

    gameweek += 1


all_gameweeks_df = pd.concat(all_gameweek_data, ignore_index=True)
all_gameweeks_df.rename(columns={'total_points': 'points'}, inplace=True)

to_drop = general_stats[general_stats["can_select"] == False]["id"].tolist()

all_gameweeks_df = all_gameweeks_df[~all_gameweeks_df["id"].isin(to_drop)]

print("All gameweek data fetched and combined.")

Fetched data for gameweek 1
Fetched data for gameweek 2
Fetched data for gameweek 3
Fetched data for gameweek 4
Fetched data for gameweek 5
Fetched data for gameweek 6
Fetched data for gameweek 7
Fetched data for gameweek 8
Fetched data for gameweek 9
Fetched data for gameweek 10
Fetched data for gameweek 11
Fetched data for gameweek 12
Fetched data for gameweek 13
Fetched data for gameweek 14
Fetched data for gameweek 15
Fetched data for gameweek 16
Fetched data for gameweek 17
Fetched data for gameweek 18
No more data available after gameweek 18
All gameweek data fetched and combined.


## 

In [40]:
all_gameweeks_df.head()

,id,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,...,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,points,in_dreamteam,gameweek
1,2,5,0,0,0,0,0,0,0,1,...,0.0,0.1,0,0.00,0.00,0.00,0.15,0,False,1
2,3,90,0,0,1,0,0,0,0,0,...,8.0,2.3,1,0.12,0.02,0.14,0.47,6,False,1
3,4,90,1,1,1,0,0,0,0,0,...,46.0,12.5,1,0.45,0.04,0.49,0.47,12,True,1
5,6,21,0,0,0,0,0,0,0,0,...,8.0,0.8,0,0.02,0.01,0.03,0.15,1,False,1
6,7,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,1


In [41]:
# Collect player total points
for i in range(1, len(all_gameweeks_df['id'].unique()) + 1):

    # Get player details
    player_details_url_formatted = player_details_url.format(element_id=i)
    player_details = requests.get(player_details_url_formatted).json()

    player_details_history_df = pd.DataFrame(player_details["history"])

    total_points = 0

    for _, row in player_details_history_df.iterrows():
        # Find matching rows in all_gameweeks_df
        condition = (all_gameweeks_df['id'] == i) & \
                    (all_gameweeks_df['gameweek'] == row['round'])

        total_points += row['total_points']

        all_gameweeks_df.loc[condition, 'total_points'] = total_points
        all_gameweeks_df.loc[condition, 'selected'] = row['selected']
        all_gameweeks_df.loc[condition, 'transfers_balance'] = row['transfers_balance']
        all_gameweeks_df.loc[condition, 'value'] = row['value']
        all_gameweeks_df.loc[condition, 'was_home'] = row['was_home']

2025-01-07 10:49:46,844 WARNING: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [42]:
all_gameweeks_df.head()

,id,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,...,expected_goal_involvements,expected_goals_conceded,points,in_dreamteam,gameweek,total_points,selected,transfers_balance,value,was_home
1,2,5,0,0,0,0,0,0,0,1,...,0.00,0.15,0,False,1,0.0,199810.0,0.0,70.0,True
2,3,90,0,0,1,0,0,0,0,0,...,0.14,0.47,6,False,1,6.0,1167503.0,0.0,60.0,True
3,4,90,1,1,1,0,0,0,0,0,...,0.49,0.47,12,True,1,12.0,1087445.0,0.0,80.0,True
5,6,21,0,0,0,0,0,0,0,0,...,0.03,0.15,1,False,1,1.0,73714.0,0.0,55.0,True
6,7,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0,False,1,0.0,25861.0,0.0,50.0,True


In [43]:
lagged_features = ['minutes', 'goals_scored', 'assists', 'clean_sheets',
                   'goals_conceded', 'own_goals', 'penalties_saved',
                   'penalties_missed', 'yellow_cards', 'red_cards', 'saves',
                   'bonus', 'bps', 'influence', 'creativity', 'threat',
                   'ict_index', 'starts', 'expected_goals', 'expected_assists',
                   'expected_goal_involvements', 'expected_goals_conceded',
                   'in_dreamteam', 'total_points', 'selected', 'transfers_balance',
                   'value', 'was_home']

for col in lagged_features:
    # Create a new column with the prefix 'prev_' containing the lagged values of the original column
    all_gameweeks_df[f'prev_{col}'] = all_gameweeks_df.groupby('id')[col].shift(1)

# Drop original columns
# Remove the original columns from the DataFrame, keeping only the lagged features
all_gameweeks_df.drop(columns=lagged_features, inplace=True)

# Drop rows with missing values in lagged features or 'points'
# Remove rows that have NaN values in any of the lagged feature columns or the 'points' column
final_data = all_gameweeks_df.dropna(subset=[f'prev_{col}' for col in lagged_features] + ['points'])

In [44]:
final_data.head()

,id,points,gameweek,prev_minutes,prev_goals_scored,prev_assists,prev_clean_sheets,prev_goals_conceded,prev_own_goals,prev_penalties_saved,...,prev_expected_goals,prev_expected_assists,prev_expected_goal_involvements,prev_expected_goals_conceded,prev_in_dreamteam,prev_total_points,prev_selected,prev_transfers_balance,prev_value,prev_was_home
617,2,0,2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.15,False,0.0,199810.0,0.0,70.0,True
618,3,6,2,90.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.12,0.02,0.14,0.47,False,6.0,1167503.0,0.0,60.0,True
619,4,2,2,90.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.45,0.04,0.49,0.47,True,12.0,1087445.0,0.0,80.0,True
621,6,6,2,21.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.02,0.01,0.03,0.15,False,1.0,73714.0,0.0,55.0,True
622,7,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,False,0.0,25861.0,0.0,50.0,True


In [45]:
final_data = final_data.astype({col: 'float64' for col in final_data.select_dtypes(include='object').columns})

In [46]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8183 entries, 617 to 11983
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               8183 non-null   int64  
 1   points                           8183 non-null   int64  
 2   gameweek                         8183 non-null   int64  
 3   prev_minutes                     8183 non-null   float64
 4   prev_goals_scored                8183 non-null   float64
 5   prev_assists                     8183 non-null   float64
 6   prev_clean_sheets                8183 non-null   float64
 7   prev_goals_conceded              8183 non-null   float64
 8   prev_own_goals                   8183 non-null   float64
 9   prev_penalties_saved             8183 non-null   float64
 10  prev_penalties_missed            8183 non-null   float64
 11  prev_yellow_cards                8183 non-null   float64
 12  prev_red_cards        

In [47]:
final_data.dropna(inplace=True)

In [48]:
fs = project.get_feature_store()

In [49]:
player_fg = fs.get_or_create_feature_group(
    name="player_features",
    description="Player data for the Fantasy Premier League",
    primary_key=["id", "gameweek"],
    version=1,
)

In [50]:
player_fg.insert(final_data)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1159321/fs/1150024/fg/1393684


Uploading Dataframe: 100.00% |██████████| Rows 8183/8183 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: player_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1159321/jobs/named/player_features_1_offline_fg_materialization/executions


(Job('player_features_1_offline_fg_materialization', 'SPARK'), None)

In [51]:
player_fg.update_feature_description("id", "Player ID")
player_fg.update_feature_description("gameweek", "Gameweek")
player_fg.update_feature_description("points", "Total points of the player in the gameweek (label)")
player_fg.update_feature_description("prev_total_points", "Total points accumulated by player up until the previous gameweek")
player_fg.update_feature_description("prev_minutes", "Played minutes in the previous gameweek")
player_fg.update_feature_description("prev_goals_scored", "Goals scored in the previous gameweek")
player_fg.update_feature_description("prev_assists", "Assists in the previous gameweek")
player_fg.update_feature_description("prev_clean_sheets", "Clean sheets in the previous gameweek")
player_fg.update_feature_description("prev_goals_conceded", "Goals conceded in the previous gameweek")
player_fg.update_feature_description("prev_own_goals", "Own goals in the previous gameweek")
player_fg.update_feature_description("prev_penalties_saved", "Penalties saved in the previous gameweek")
player_fg.update_feature_description("prev_penalties_missed", "Penalties missed in the previous gameweek")
player_fg.update_feature_description("prev_yellow_cards", "Yellow cards in the previous gameweek")
player_fg.update_feature_description("prev_red_cards", "Red cards in the previous gameweek")
player_fg.update_feature_description("prev_saves", "Saves in the previous gameweek")
player_fg.update_feature_description("prev_bonus", "Bonus points in the previous gameweek")
player_fg.update_feature_description("prev_bps", "Bonus points system in the previous gameweek")
player_fg.update_feature_description("prev_influence", "Influence in the previous gameweek")
player_fg.update_feature_description("prev_creativity", "Creativity in the previous gameweek")
player_fg.update_feature_description("prev_threat", "Threat in the previous gameweek")
player_fg.update_feature_description("prev_ict_index", "ICT index in the previous gameweek")
player_fg.update_feature_description("prev_starts", "Player started in the previous gameweek")
player_fg.update_feature_description("prev_expected_goals", "Expected goals in the previous gameweek")
player_fg.update_feature_description("prev_expected_assists", "Expected assists in the previous gameweek")
player_fg.update_feature_description("prev_expected_goal_involvements", "Expected goal involvements in the previous gameweek")
player_fg.update_feature_description("prev_expected_goals_conceded", "Expected goals conceded in the previous gameweek")
player_fg.update_feature_description("prev_selected", "Amount of players that selected the player in the previous gameweek")
player_fg.update_feature_description("prev_transfers_balance", "Sum of transfers in and out in the previous gameweek")
player_fg.update_feature_description("prev_value", "Value of the player up until the previous gameweek")
player_fg.update_feature_description("prev_was_home", "Whether the player played at home in the previous gameweek")